In [ ]:
from langchain_google_community.calendar.delete_event import CalendarDeleteEvent
from langchain_google_community.calendar.create_event import CalendarCreateEvent
from langchain_google_community.calendar.search_events import CalendarSearchEvents
from langchain_google_community.calendar.get_calendars_info import GetCalendarsInfo



tool1=CalendarDeleteEvent()
tool2=CalendarCreateEvent()
tool3=CalendarSearchEvents()
tool4=GetCalendarsInfo()

tools=[tool1,tool2,tool3,tool4]

In [ ]:
import getpass
import os
 

os.environ["AZURE_OPENAI_ENDPOINT"] = "https://openai0perks.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview"

from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_deployment="gpt-4o",  # or your deployment
    api_version="2025-01-01-preview",  # or your api version
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key='Your openai key'
    # other params...
)

In [23]:
from langgraph.prebuilt import create_react_agent

graph=create_react_agent(llm,tools)

In [ ]:
def parse_response(stream):
    tool_called_name='None'
    final_response=None
    
    for s in stream:
        tool_data=s.get('tools')
        if tool_data:
            tool_messages=tool_data.get('messages')
            if tool_messages and isinstance(tool_messages, list):
                for msg in tool_messages:
                    tool_called_name=getattr(msg,'name','None')
        
        agent_data=s.get('agent')
        if agent_data:
            messages=agent_data.get('messages')
            if messages and isinstance(messages, list):
                for msg in messages:
                    if msg.content:
                        final_response=msg.content
    return tool_called_name, final_response
inputs={'messages': [ ('user', 'can you please create an event on 12 august 2025 at 7pm to 8pm with the name of study location is lahore and there is no description')]}
stream=graph.stream(inputs, stream_mode='updates')
tool_called_name, final_response=parse_response(stream)
print('Tool called: ',tool_called_name)
print('final response: ',final_response)

Tool called:  create_event
final response:  The event "Study" has been successfully created on August 12, 2025, from 7:00 PM to 8:00 PM at Lahore. Let me know if you need any further assistance!
